# Load CSV file into neo4j database

First using commands taken from previous labs.

In [1]:
import neo4j

import pandas as pd

from IPython.display import display

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

# Loading CSV File into Neo4j

go to https:// **IP here** :7473 for web browser

Data info

|variable          |class     |description       |
|:-----------------|:---------|:-----------------|
|year              |double    |The year.              |
|coo_name          |character |Country of origin name.        |
|coo               |character |Country of origin UNHCR code.   |
|coo_iso           |character |Country of origin ISO code.  |
|coa_name          |character |Country of asylum name.    |
|coa               |character |Country of asylum UNHCR code.  |
|coa_iso           |character |Country of asylum ISO code.    |
|refugees          |double    |The number of refugees.   |
|asylum_seekers    |double    |The number of asylum-seekers.  |
|returned_refugees |double    |The number of returned refugees. |
|idps              |double    |The number of internally displaced persons.     |
|returned_idps     |double    |The number of returned internally displaced persons.  |
|stateless         |double    |The number of stateless persons.  |
|ooc               |double    |The number of others of concern to UNHCR.   |
|oip               |double    |The number of other people in need of international protection.     |
|hst               |double    |The number of host community members.     |

In [7]:
df = pd.read_csv("data/population2010.csv")
df.head()

,year,coo_name,coo,coo_iso,coa_name,coa,coa_iso,refugees,asylum_seekers,returned_refugees,idps,returned_idps,stateless,ooc,oip,hst
0,2010,Afghanistan,AFG,AFG,Afghanistan,AFG,AFG,0,0,0,351907,3366,0,838250,NaN,0
1,2010,Iran (Islamic Rep. of),IRN,IRN,Afghanistan,AFG,AFG,30,21,0,0,0,0,0,NaN,0
2,2010,Iraq,IRQ,IRQ,Afghanistan,AFG,AFG,6,0,0,0,0,0,0,NaN,0
3,2010,Pakistan,PAK,PAK,Afghanistan,AFG,AFG,6398,9,0,0,0,0,0,NaN,0
4,2010,Egypt,ARE,EGY,Albania,ALB,ALB,5,0,0,0,0,0,0,NaN,0


## Example Cypher Queries

In [8]:
# load the data in as needed.
# only keep relationships with >= 1 to consolidate, can change as needed.

my_neo4j_wipe_out_database()

query = """

LOAD CSV WITH HEADERS FROM 'file:///population2010.csv' AS row
MERGE (origin:Country {iso_code: row.coo_iso, name: row.coo_name})
MERGE (asylum:Country {iso_code: row.coa_iso, name: row.coa_name})

WITH row, origin, asylum
WHERE toInteger(row.refugees) >= 1
CREATE (origin)-[:REFUGEE_FLOW {number: toInteger(row.refugees)}]->(asylum)
"""

session.run(query)

In [9]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,Afghanistan,[Country]
1,Albania,[Country]
2,Algeria,[Country]
3,Andorra,[Country]
4,Angola,[Country]
...,...,...
190,Viet Nam,[Country]
191,Western Sahara,[Country]
192,Yemen,[Country]
193,Zambia,[Country]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,Afghanistan,[Country],REFUGEE_FLOW,Australia,[Country]
1,Afghanistan,[Country],REFUGEE_FLOW,Austria,[Country]
2,Afghanistan,[Country],REFUGEE_FLOW,Azerbaijan,[Country]
3,Afghanistan,[Country],REFUGEE_FLOW,Belarus,[Country]
4,Afghanistan,[Country],REFUGEE_FLOW,Belgium,[Country]
...,...,...,...,...,...
3471,Zimbabwe,[Country],REFUGEE_FLOW,Sweden,[Country]
3472,Zimbabwe,[Country],REFUGEE_FLOW,Switzerland,[Country]
3473,Zimbabwe,[Country],REFUGEE_FLOW,United Kingdom of Great Britain and Northern I...,[Country]
3474,Zimbabwe,[Country],REFUGEE_FLOW,United States of America,[Country]


-------------------------
  Density: 0.2
-------------------------


In [32]:
query = """

MATCH (n)-[r]->(b)
WHERE n.name CONTAINS 'Afghanistan'
RETURN n.name AS origin_name, 
       n.iso_code AS origin_iso_code,
       toInteger(r.number) AS refugee_number, 
       b.name AS asylum_name,
       b.iso_code AS asylum_iso_code

ORDER BY refugee_number DESC
LIMIT 5

"""

my_neo4j_run_query_pandas(query)

,origin_name,origin_iso_code,refugee_number,asylum_name,asylum_iso_code
0,Afghanistan,AFG,1899842,Pakistan,PAK
1,Afghanistan,AFG,1027577,Iran (Islamic Rep. of),IRN
2,Afghanistan,AFG,30404,Germany,DEU
3,Afghanistan,AFG,23406,United Kingdom of Great Britain and Northern I...,GBR
4,Afghanistan,AFG,9094,India,IND
